# Inverse kinematics

An example of how tlm can be used to solve a simple 3D inverse kinematics problem.

In [ ]:
import torch
import torch.nn as nn
import torchlensmaker as tlm

y1 = tlm.parameter(torch.tensor(-20))
z1 = tlm.parameter(torch.tensor(0))

y2 = tlm.parameter(torch.tensor(0))
z2 = tlm.parameter(torch.tensor(0))

length1 = tlm.parameter(10.)

class Target(tlm.SequentialElement):
    def __init__(self, point):
        super().__init__()
        self.point = point

    def forward(self, inputs):
        return inputs.replace(loss=torch.linalg.vector_norm(inputs.target() - self.point))

model = tlm.Sequential(
    tlm.Gap(length1),
    tlm.Rotate3D(y1, z1),
    tlm.Gap(5),
    tlm.Rotate3D(y2, z2),
    tlm.Gap(5),
    Target(torch.Tensor([20, 6, 6])),
)

for name, param in model.named_parameters():
    print(name, param)

tlm.show3d(model)

In [ ]:
import torch.optim as optim


tlm.optimize(
    model,
    optimizer = optim.Adam(model.parameters(), lr=0.5),
    sampling = {},
    dim = 3,
    num_iter = 100
).plot()

print("length:", length1.item())
print("y1:", torch.rad2deg(y1).detach().numpy())
print("z1:", torch.rad2deg(z1).detach().numpy())
print("y2:", torch.rad2deg(y2).detach().numpy())
print("z2:", torch.rad2deg(z2).detach().numpy())

tlm.show3d(model)